This notebook is a continuouation of LSTM Model creation (v4), you shall use the part 1 of LSTM Model v3 and continue as done in V3 model creation notebook. upload the created folder in part 1 manually to your google drive, and follow below steps with necessary adjustments in order to have LSTM Model V4

below LSTM model had very low scores, further enhancements can be done to raise the overall performance

this notebook can only run in Coolab Pro utilizing the available TPU
### main differences are: class weights were created, and validation data frame was deleted, overall score is low

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
import os
import tensorflow as tf
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)



In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#extract the LSTM_Seq zip file

from google.colab import drive
import os
import zipfile


# Navigate to your uploaded file location
drive_path = '/content/drive/My Drive/Colab Notebooks/LSTM_Seq.zip'  # Adjust this path to match your upload
output_dir = '/content/LSTM_Seq/'

# Extract the zip file
os.makedirs(output_dir, exist_ok=True)
with zipfile.ZipFile(drive_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Files extracted to {output_dir}")


KeyboardInterrupt: 

In [ ]:
import os

nested_dir = "/content/LSTM_Seq/LSTM_Seq"
if os.path.exists(nested_dir):
    print(f"Files in {nested_dir}:")
    print(os.listdir(nested_dir))
else:
    print("No nested directory found.")


In [3]:
import shutil

nested_dir = "/content/LSTM_Seq/LSTM_Seq"
output_dir = "/content/LSTM_Seq"

# Move all files from nested_dir to output_dir
if os.path.exists(nested_dir):
    for file_name in os.listdir(nested_dir):
        shutil.move(os.path.join(nested_dir, file_name), output_dir)

    # Remove the now-empty nested directory
    shutil.rmtree(nested_dir)

print(f"Files after moving:")
print(os.listdir(output_dir))


Error: Destination path '/content/LSTM_Seq/train_X_14966000.npy' already exists

In [4]:
import numpy as np
import glob
import os


def load_batches_from_dir(output_dir, output_prefix):
    """
    Load all .npy files with the given prefix from the specified directory.
    """
    X_files = sorted(glob.glob(os.path.join(output_dir, f"{output_prefix}_X_*.npy")))
    y_files = sorted(glob.glob(os.path.join(output_dir, f"{output_prefix}_y_*.npy")))

    X = np.concatenate([np.load(f) for f in X_files], axis=0)
    y = np.concatenate([np.load(f) for f in y_files], axis=0)

    return X, y

# Load train and test datasets
output_dir = "/content/LSTM_Seq/"
X_train, y_train = load_batches_from_dir(output_dir, "train")
X_test, y_test = load_batches_from_dir(output_dir, "test")

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (18096012, 60, 5), y_train shape: (18096012,)
X_test shape: (3973708, 60, 5), y_test shape: (3973708,)


In [5]:
#step 4
from tensorflow.keras.utils import to_categorical

# Map class labels to zero-indexed values
unique_classes = sorted(np.unique(y_train))  # Find unique classes in sorted order
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}  # Map original class to new index

# Apply the mapping
y_train_mapped = np.array([class_mapping[label] for label in y_train])
y_test_mapped = np.array([class_mapping[label] for label in y_test])

# One-hot encode the remapped labels
num_classes = len(unique_classes)  # Total number of unique classes
y_train_encoded = to_categorical(y_train_mapped, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_mapped, num_classes=num_classes)

# Verify the shapes
print(f"y_train_encoded shape: {y_train_encoded.shape}, y_test_encoded shape: {y_test_encoded.shape}")
print(f"Class mapping: {class_mapping}")


y_train_encoded shape: (18096012, 4), y_test_encoded shape: (3973708, 4)
Class mapping: {0: 0, 101: 1, 105: 2, 107: 3}


In [6]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
    print(f"Running on TPU: {tpu.master()}")
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)  # Create a strategy
except ValueError as e:
    print("TPU not found. Falling back to CPU/GPU.")
    print("Error:", e)
    strategy = tf.distribute.get_strategy()

print("Number of replicas in sync:", strategy.num_replicas_in_sync)


Running on TPU: 
Number of replicas in sync: 8


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.mixed_precision import set_global_policy
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Reset precision policy to float32 for TPU compatibility
set_global_policy('float32')

# Verify the current precision policy
print("Current precision policy:", tf.keras.mixed_precision.global_policy())

# Validate the datasets
print(f"X_train shape: {X_train.shape}, y_train_encoded shape: {y_train_encoded.shape}")
print(f"X_test shape: {X_test.shape}, y_test_encoded shape: {y_test_encoded.shape}")

# Compute class weights
unique_classes = np.unique(y_train_mapped)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train_mapped)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Computed Class Weights:", class_weights_dict)

# Use TensorFlow Dataset for efficient data pipeline
batch_size = 1500  # Adjust based on TPU capabilities
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded))
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test_encoded))
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# TPU setup
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
    print('Running on TPU:', tpu.master())

    # Initialize only if TPU is not already initialized
    if not tf.config.list_logical_devices('TPU'):
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.TPUStrategy(tpu)  # Create a strategy
except ValueError as e:
    print("TPU not found. Falling back to CPU/GPU.")
    print("Error:", e)
    strategy = tf.distribute.get_strategy()

print("Number of replicas in sync:", strategy.num_replicas_in_sync)



Current precision policy: <Policy "float32">
X_train shape: (18096012, 60, 5), y_train_encoded shape: (18096012, 4)
X_test shape: (3973708, 60, 5), y_test_encoded shape: (3973708, 4)
Computed Class Weights: {0: 0.7851000125470144, 1: 1.1048112004923294, 2: 2.281798382768903, 3: 0.7231211769029111}
Running on TPU: 
Number of replicas in sync: 8


In [ ]:
# Define the LSTM model within the TPU strategy scope
with strategy.scope():
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for efficient training
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model_tpu.h5', monitor='loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-5)

# Train the model
history = model.fit(
    train_dataset,
    epochs=50,
    class_weight=class_weights_dict,  # Add class weights
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

# Save the final model
model.save("final_model_tpu.h5")


In [9]:
from tensorflow.keras.models import load_model

# Load the best saved model
best_model = load_model('best_model_tpu.h5')

# Evaluate the best model
best_test_loss, best_test_accuracy = best_model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Best Model - Test Loss: {best_test_loss:.4f}, Test Accuracy: {best_test_accuracy:.4f}")


# Load the final model
final_model = load_model('final_model_tpu.h5')  # Replace 'final_model_tpu.h5' with your actual file name

# Evaluate the final model
final_test_loss, final_test_accuracy = final_model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Final Model - Test Loss: {final_test_loss:.4f}, Test Accuracy: {final_test_accuracy:.4f}")

Best Model - Test Loss: 1.1992, Test Accuracy: 0.7719
Final Model - Test Loss: 1.1992, Test Accuracy: 0.7719


In [10]:
import tensorflow as tf

# Define the batch size for evaluation
batch_size = 2048  # Adjust this value based on memory availability

# Create the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test_encoded))
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [11]:
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import classification_report

# Load the best saved model
best_model = load_model('best_model_tpu.h5')

# Predict with the best saved model
y_pred_probs_best = best_model.predict(test_dataset, verbose=0)
y_pred_best = np.argmax(y_pred_probs_best, axis=1)

# Load the final model (if it is saved after training)
try:
    final_model = load_model('final_model_tpu.h5')  # Ensure this file exists
    y_pred_probs_final = final_model.predict(test_dataset, verbose=0)
    y_pred_final = np.argmax(y_pred_probs_final, axis=1)
except OSError:
    print("Final model file not found. Ensure 'final_model_tpu.h5' exists.")
    y_pred_probs_final = None
    y_pred_final = None

# Get true labels from the test dataset
y_true = np.argmax(y_test_encoded, axis=1)

# Print shapes for verification
print(f"True Labels Shape: {y_true.shape}")
print(f"Best Model Predictions Shape: {y_pred_best.shape}")
if y_pred_final is not None:
    print(f"Final Model Predictions Shape: {y_pred_final.shape}")

# Classification Report for the Best Model
print("\nClassification Report for Best Model:")
print(classification_report(y_true, y_pred_best, target_names=[str(cls) for cls in unique_classes]))

# Classification Report for the Final Model
if y_pred_final is not None:
    print("\nClassification Report for Final Model:")
    print(classification_report(y_true, y_pred_final, target_names=[str(cls) for cls in unique_classes]))
else:
    print("\nFinal model predictions are unavailable.")


True Labels Shape: (3973708,)
Best Model Predictions Shape: (3973708,)
Final Model Predictions Shape: (3973708,)

Classification Report for Best Model:
              precision    recall  f1-score   support

           0       0.71      0.72      0.71   1369753
           1       0.85      0.91      0.88   1164386
           2       0.58      0.11      0.18    440719
           3       0.78      0.98      0.87    998850

    accuracy                           0.77   3973708
   macro avg       0.73      0.68      0.66   3973708
weighted avg       0.75      0.77      0.74   3973708


Classification Report for Final Model:
              precision    recall  f1-score   support

           0       0.71      0.72      0.71   1369753
           1       0.85      0.91      0.88   1164386
           2       0.58      0.11      0.18    440719
           3       0.78      0.98      0.87    998850

    accuracy                           0.77   3973708
   macro avg       0.73      0.68      0.66   3

In [13]:
# Staking ..

# Step 2: Train additional models using LSTM predictions as meta-features
print("Training additional models for stacking...")

# Use LSTM predictions as input features for stacking models
X_train_meta = best_model.predict(X_train, verbose=1)  # Meta-features for training
X_test_meta = y_pred_probs_best  # Meta-features for testing (from the best model predictions)



Training additional models for stacking...
565501/565501 [==============================] - 7381s 13ms/step


NameError: name 'RandomForestClassifier' is not defined

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
# Random Forest
print("Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train_meta, y_train_mapped)
print("Predicting probabilities with Random Forest...")
y_test_probs_rf = rf_model.predict_proba(X_test_meta)

# Logistic Regression
print("Training Logistic Regression...")
lr_model = LogisticRegression(class_weight='balanced', max_iter=1000, verbose=1)
lr_model.fit(X_train_meta, y_train_mapped)
print("Predicting probabilities with Logistic Regression...")
y_test_probs_lr = lr_model.predict_proba(X_test_meta)

# Step 3: Combine predictions from all models
print("Combining predictions using stacking...")
y_test_probs_combined = (y_pred_probs_best + y_test_probs_rf + y_test_probs_lr) / 3
y_test_stacked = np.argmax(y_test_probs_combined, axis=1)

# Step 4: Evaluate the stacked model
y_true = y_test_mapped  # True labels

print("\nClassification Report for Stacked Model:")
print(classification_report(y_true, y_test_stacked, target_names=[str(cls) for cls in unique_classes]))

# Optional: Save the stacked model predictions for further analysis
np.save("stacked_predictions.npy", y_test_probs_combined)

Training Random Forest...
Predicting probabilities with Random Forest...
Training Logistic Regression...
Predicting probabilities with Logistic Regression...
Combining predictions using stacking...

Classification Report for Stacked Model:
              precision    recall  f1-score   support

           0       0.92      0.73      0.81   1369753
           1       0.81      0.92      0.86   1164386
           2       0.91      0.78      0.84    440719
           3       0.78      0.94      0.85    998850

    accuracy                           0.84   3973708
   macro avg       0.86      0.84      0.84   3973708
weighted avg       0.85      0.84      0.84   3973708

